In [10]:
import pandas as pd
import pathlib
from pathlib import Path
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [2]:
def calc_stake(odds, win_prob):
    if odds > 0:
        decimal = (odds / 100) + 1
    else:
        decimal = (100 / abs(odds)) + 1
        
    stake = (((decimal * win_prob[1]) - 1) / (decimal - 1)) / 10
    if stake < .02:
        return stake
    else:
        return .02

In [3]:
def calc_payout(odds, wager):
    if odds < 0:
        payout = (100 / abs(odds)) * wager
    else:
        payout = (odds / 100) * wager
        
    return payout

In [4]:
def calc_ev(odds, win_prob, wager):
    payout = calc_payout(odds, wager)
    
    return (payout * win_prob[1]) - (wager * win_prob[0])

In [13]:
data = pd.read_csv(str(Path.cwd()) + '/final_updated.csv')

In [14]:
data

,Date,GameID,Season,H/A,Team,ML,Wins,WinLast10,Elo,EloLast10,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXH/A,XXTeam,XXML,XXWins,XXWinLast10,XXElo,XXEloLast10,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBall

In [15]:
data = pd.read_csv(str(Path.cwd()) + '/final_updated.csv')
data.drop(['Date', 'GameID', 'TeamID', 'XXTeamID', 'Team', 'XXTeam', 'Temp', 'XXTemp'], axis=1, inplace=True)
seasons = data.Season.unique()
data = [data.loc[data.Season == season] for season in seasons]
for idx, df in enumerate(data):
    data[idx] = df.drop(['Season'], axis=1)

In [7]:
X = [df.loc[df['OffPoss'] != 0] for df in data]
for idx, df in enumerate(X):
    X[idx] = df.dropna(axis=0)
    
y = [df.W_L for df in X]
X = [df.drop(['W_L', 'XXW_L'], axis=1) for df in X]

In [8]:
X_50 = [df[len(df)//2:] for df in X]
y_50 = [df[len(df)//2:] for df in y]
X_50_ = [df[:len(df)//2] for df in X]
y_50_ = [df[:len(df)//2] for df in y]

for idx, df in enumerate(y_50):
    y_50[idx] = df.reset_index(drop=True)
    X_50[idx] = X_50[idx].reset_index(drop=True)

In [12]:
tests = []
for kernel in tqdm(['poly', 'rbf', 'linear']):
    if kernel == 'linear':
        c_vals = [2**-6, 2**-5.5, 2**-5, 2**-4.5, 2**-4, 2**-3, 2**-2, 2**-1, 1, 2, 2**2, 2**3, 2**4, 2**6]
    elif kernel == 'poly':
        c_vals = [2**3, 9, 10, 11, 12, 13, 14, 15, 16]
    else:
        c_vals = [16 + i for i in range(49)] + [2**8, 260, 264, 300]
    for c in c_vals:
        scores = []
        print('C: ' + str(c) + '| Kernel: ' + kernel)
        print('-----------------------------')
        for idx, df in enumerate(X):
            X_data = pd.concat([X[i] for i in range(len(X)) if i != idx], axis=0)
            y_data = pd.concat([y[i] for i in range(len(y)) if i != idx], axis=0)
            clf = svm.SVC(kernel=kernel, C=c, random_state=0)
            clf.fit(pd.concat([X_data, X_50_[idx]],axis=0), pd.concat([y_data, y_50_[idx]], axis=0))
            print(clf.score(X_50[idx], y_50[idx]))
            scores.append(clf.score(X_50[idx], y_50[idx]))
            
        scores = sorted(scores)
        tests.append({'kernel': kernel, 'c': c, 'avg_score': sum(scores) / 9,
                      'low_score': scores[0], 'high_score': scores[8]})
        print('-----------------------------')
        print('Average Score: ' + str(sum(scores) / 9))
        print('-----------------------------')

  0%|                                                     | 0/3 [00:00<?, ?it/s]

C: 8| Kernel: poly
-----------------------------


  0%|                                                     | 0/3 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
tests_df = pd.DataFrame(tests)

In [ ]:
tests_df.loc[tests_df['avg_score'] > .69]

In [9]:
coef_vals = []
for idx, df in enumerate(X):
    X_data = pd.concat([X[i] for i in range(len(X)) if i != idx], axis=0)
    y_data = pd.concat([y[i] for i in range(len(y)) if i != idx], axis=0)
    clf = svm.SVC(kernel='linear', C=.0625, random_state=0)
    clf.fit(pd.concat([X_data, X_50_[idx]],axis=0), pd.concat([y_data, y_50_[idx]], axis=0))
    coef_vals.append(clf.coef_)

In [10]:
averages = [
    sum([coef_vals[0][0][i], coef_vals[1][0][i], coef_vals[2][0][i],
         coef_vals[3][0][i], coef_vals[4][0][i], coef_vals[5][0][i],
         coef_vals[6][0][i], coef_vals[7][0][i], coef_vals[8][0][i]]) / 9 for i in range(len(coef_vals[0][0]))
]

averages = np.array(averages)
features = np.array(X[0].columns)
sorted_idx = averages.argsort()

In [11]:
feature_tests = []
for kernel in ['poly', 'rbf']:
    if kernel == 'poly':
        c, nums  = 11, [124 + (2 * i) for i in range(8)]
    else:
        c, nums = 29, [164 + (2 * i) for i in range(8)]
    for num in tqdm(nums):
        feat_subset = features[sorted_idx][:num]
        scores = []
        print('C: ' + str(c) + ' | Kernel: ' + kernel + ' | Features: ' + str(num))
        print('-----------------------------')
        for idx, df in enumerate(X):
            clf = svm.SVC(kernel=kernel, C=c, random_state=0)
            clf.fit(pd.concat([X_data[feat_subset], X_50_[idx][feat_subset]],axis=0), pd.concat([y_data, y_50_[idx]], axis=0))
            scores.append(clf.score(X_50[idx][feat_subset], y_50[idx]))
            print(clf.score(X_50[idx][feat_subset], y_50[idx]))

        scores = sorted(scores)
        feature_tests.append({'kernel': kernel, 'c': c, 'avg_score': sum(scores) / 9,
                      'low_score': scores[0], 'high_score': scores[8], 'features': num})
        print('-----------------------------')
        print('Average Score: ' + str(sum(scores) / 9))
        print('-----------------------------')

  0%|                                                     | 0/8 [00:00<?, ?it/s]


NameError: name 'features' is not defined

In [ ]:
feat_test_df = pd.DataFrame(feature_tests)

In [ ]:
feat_test_df

In [ ]:
feature_list = []
for idx in range(9):
    clf = svm.SVC(kernel=kernel, C=c, random_state=0)
    perm_importance = permutation_importance(clf, pd.concat([X_data, X_50_[idx]],axis=0), pd.concat([y_data, y_50_[idx]], axis=0))
    print('Season: ' + idx + 'Done')
    features = np.array(X_data.columns)
    sorted_idx = perm_importance.importances_mean.argsort()
    features_list.append(features[sorted_idx][:160])

In [12]:
for kernel in ['poly', 'rbf']:
    if kernel == 'poly':
        nums = [124 + (2 * i) for i in range(8)]
        c_vals = [10, 11, 12, 13, 14]
    else:
        nums = [164 + (2 * i) for i in range(8)]
        c_vals = [28, 29, 30, 31, 32]
        
    for num in nums:
        feat_subset = features[sorted_idx][:num]
        for c in c_vals:
            payout = []
            output = []
            probs = []
            for idx, df in enumerate(X_50):
                clf = svm.SVC(kernel=kernel, C=c, probability=True, random_state=0)
                X_data = pd.concat([X[i] for i in range(len(X)) if i != idx], axis=0)
                y_data = pd.concat([y[i] for i in range(len(y)) if i != idx], axis=0)
                clf.fit(pd.concat([X_data[feat_subset], X_50_[idx][feat_subset]],axis=0), pd.concat([y_data, y_50_[idx]], axis=0))
                probs.append(clf.predict_proba(X_50[idx][feat_subset]))
                output.append([calc_ev(row, probs[idx][i], 100) for i, row in enumerate(df.ML)])

            bank = 2500
            print('------------------------------------')
            print('Kernel: ' + kernel + ' | C: ' + str(c) + ' | Features: ' + str(num))
            print('------------------------------------')
            for idx, season in enumerate(output):
                count = 0
                total = 0
                for i, game in enumerate(season):
                    count += 1
                    stake = bank * calc_stake(X_50[idx]['ML'].loc[i], probs[idx][i])
                    if bank == 0:
                        stake = 0

                    if ((game > 3) & (y_50[idx][i] == 1) & (stake > 0)):
                        total += calc_payout(X_50[idx]['ML'].loc[i], stake)
                        payout.append(
                            {'pay': calc_payout(X_50[idx]['ML'].loc[i], stake), 'season': idx, 'game': count, 'total': total, 'odds': X_50[idx]['ML'].loc[i], 'bank': bank, 'stake': stake})
                    elif ((game > 3) & (y_50[idx][i] == 0) & (stake > 0)):
                        total += -1 * stake
                        payout.append({'pay': -1 * stake, 'season': idx, 'game': count, 'total': total, 'odds': X_50[idx]['ML'].loc[i], 'bank': bank, 'stake': stake})

                bank += total    
                print('Season Total: ' + str(total))

            print('Profit: ' + str(bank - 2500))

------------------------------------
Kernel: poly | C: 10 | Features: 124
------------------------------------
Season Total: -753.6850000652548
Season Total: 629.2356877963712
Season Total: -222.72844536010848
Season Total: 698.9088696913523
Season Total: -1123.4741547702358
Season Total: -279.89196553005
Season Total: 146.79467311070115
Season Total: -27.478976926583847
Season Total: -1048.4405373769912
Profit: -1980.7598494307997
------------------------------------
Kernel: poly | C: 11 | Features: 124
------------------------------------
Season Total: -736.2051964012595
Season Total: 638.1324730695964
Season Total: -211.03112573135257
Season Total: 717.1104170963125
Season Total: -1159.0404548175975
Season Total: -284.85661385522553
Season Total: 147.89275892478145
Season Total: -13.011316723527411
Season Total: -1055.0971272704867
Profit: -1956.1061857087584
------------------------------------
Kernel: poly | C: 12 | Features: 124
------------------------------------
Season Total: 

KeyboardInterrupt: 